In [1]:
#Spam Classifier

In [1]:
#Download the data 
import os
import tarfile
import urllib 

DOWNLOAD_ROOT='https://spamassassin.apache.org/old/publiccorpus/'
HAM_URL=DOWNLOAD_ROOT + '20030228_easy_ham.tar.bz2'
SPAM_URL=DOWNLOAD_ROOT + '20030228_spam.tar.bz2'
SPAM_PATH=os.path.join('datasets','spam')

def fetch_spam_data(spam_url=SPAM_URL,spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (("ham.tar.bz2", HAM_URL), ("spam.tar.bz2", SPAM_URL)):
        file_path=os.path.join(spam_path + filename)
        if not os.path.isfile(file_path):
            urllib.request.urlretrieve(url,file_path)
        tar_bz2_file=tarfile.open(file_path)   
        tar_bz2_file.extractall(path=SPAM_PATH) 
        tar_bz2_file.close()

In [1]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT='https://spamassassin.apache.org/old/publiccorpus/'
HAM_URL=DOWNLOAD_ROOT + '20030228_easy_ham.tar.bz2'
SPAM_URL=DOWNLOAD_ROOT + '20030228_spam.tar.bz2'
SPAM_PATH=os.path.join('datasets','spam')

In [2]:
#fetch_spam_data()

In [3]:
#Extracting all ham and spam filenames
ham_dir=os.path.join(SPAM_PATH, 'easy_ham')
spam_dir=os.path.join(SPAM_PATH, 'spam')

ham_filenames=[name for name in sorted(os.listdir(ham_dir)) if len(name) > 20]
spam_filenames=[name for name in sorted(os.listdir(spam_dir)) if len(name) > 20]

In [4]:
len(ham_filenames)

2500

In [5]:
len(spam_filenames)

500

In [6]:
#Email Parser Function -
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory='spam' if is_spam else 'easy_ham'
    with open(os.path.join(spam_path,directory,filename),'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [7]:
#The output is a list of emails
ham_emails=[load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails=[load_email(is_spam=True, filename=name) for name in spam_filenames]

In [10]:
#Check the content of an email
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [11]:
ham_emails[1]

In [12]:
print(spam_emails[1].get_content().strip())

1) Fight The Risk of Cancer!
http://www.adclick.ws/p.cfm?o=315&s=pk007

2) Slim Down - Guaranteed to lose 10-12 lbs in 30 days
http://www.adclick.ws/p.cfm?o=249&s=pk007

3) Get the Child Support You Deserve - Free Legal Advice
http://www.adclick.ws/p.cfm?o=245&s=pk002

4) Join the Web's Fastest Growing Singles Community
http://www.adclick.ws/p.cfm?o=259&s=pk007

5) Start Your Private Photo Album Online!
http://www.adclick.ws/p.cfm?o=283&s=pk007

Have a Wonderful Day,
Offer Manager
PrizeMama













If you wish to leave this list please use the link below.
http://www.qves.com/trim/?ilug@linux.ie%7C17%7C114258


-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie


In [13]:
#A lot of emails are of multiple parts so we need to seggregate the emails in one simple email
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()        
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([get_email_structure(sub_email) for sub_email in payload]))  #Joining all the subparts of the email
    else:
        return email.get_content_type()

In [14]:
#Create a counter to see the different sub-parts of emails and which email contains what number of sub-parts
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [15]:
structures_counter(ham_emails)                    #Observation of the ham emails - a lot of emails are plaintext emails

Counter({'text/plain': 2408,
         'multipart(text/plain, application/pgp-signature)': 66,
         'multipart(text/plain, text/html)': 8,
         'multipart(text/plain, text/enriched)': 1,
         'multipart(text/plain, application/ms-tnef, text/plain)': 1,
         'multipart(text/plain)': 3,
         'multipart(text/plain, application/octet-stream)': 2,
         'multipart(text/plain, text/plain)': 4,
         'multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)': 1,
         'multipart(text/plain, video/mng)': 1,
         'multipart(text/plain, multipart(text/plain))': 1,
         'multipart(text/plain, application/x-pkcs7-signature)': 1,
         'multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)': 1,
         'multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))': 1,
         'multipart(text/plain, application/x-java-applet)': 1})

In [16]:
structures_counter(spam_emails)                       #A lot of email are htmls

Counter({'text/html': 183,
         'text/plain': 218,
         'multipart(text/plain, application/octet-stream)': 1,
         'multipart(text/html)': 20,
         'multipart(text/plain, text/html)': 45,
         'multipart(text/plain)': 19,
         'multipart(text/html, text/plain)': 1,
         'multipart(text/html, application/octet-stream)': 2,
         'multipart(multipart(text/html))': 5,
         'multipart(text/plain, image/jpeg)': 3,
         'multipart(multipart(text/html), application/octet-stream, image/jpeg)': 1,
         'multipart(multipart(text/plain, text/html), image/gif)': 1,
         'multipart/alternative': 1})

In [17]:
#Exploring the header of the spam email - items() function is used to extract each element in the list
for hdr, value in spam_emails[0].items():
    print(hdr,':',value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

In [18]:
spam_emails[0]['Subject']                  #We can explore each section of the email by calling out the name of the element, like we stated 'Subject' in this column

'Life Insurance - Why Pay More?'

In [19]:
spam_emails[0]['From']                 #Just to validate I have tried to display the From field of the email in this block

'12a1mailbot1@web.de'

In [118]:
#Dividing the data set in train and test data
import numpy as np
from sklearn.model_selection import train_test_split

X=np.array(ham_emails + spam_emails)
y=np.array([0]*len(spam_emails) + [1]*len(ham_emails))

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [119]:
#Extract the plain text from the email - remove the html tags
from bs4 import BeautifulSoup

In [120]:
def html_to_plain_text(html):
  
    soup = BeautifulSoup(html, "html.parser")
    return '\n'.join(soup.stripped_strings)        

In [121]:
X_train[0].get_content()

'Chuck Murcko wrote:\n>[...stuff...]\n\nYawn.\n\nR\n\n\n'

In [122]:
#The emails of type text/html contain a lot of html details, below is a spam email's content 
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[0]
print(sample_html_spam.get_content().strip()[:1000], "...")                    #Printing first 1000 characters of the email which is of type text/html

<HTML>
<BODY BGCOLOR="#ffffff">
<P>
<<HTML>
<TABLE WIDTH=400 BORDER=0 CELLPADDING=0 CELLSPACING=0>
  <TR>
    <TD ALIGN="LEFT" VALIGN="TOP"><FONT FACE="Tahoma, Arial, Verdana" SIZE=2></FONT>
      <H2>
	<FONT COLOR="#FF0000">GET HIGH...LEGALLY!</FONT>
      </H2>
      <P>
      <B>IT REALLY WORKS!<BR>
      PASSES ALL DRUG TESTS!<BR>
      EXTREMELY POTENT!</B>
      <P>
      <A HREF="http://www.greenmatrix.net/herb/index.html"><B>CLICK HERE for more
      info on Salvia Divinorum</B></A>
      <P>
      <B> <A HREF="http://www.greenmatrix.net/herb/5x.html">CLICK HERE for SALVIA
      5X EXTRACT!</A> <BR>
      <FONT COLOR="#FF0000"><BIG>WARNING...VERY POTENT!</BIG></FONT></B>
      <P>
      <B> <A HREF="http://www.greenmatrix.net/herb/13x.html">CLICK HERE for SALVIA
      13X</A>. The most POTENT, LEGAL, SMOKABLE herb on the planet! 13 times the
      power of Salvia Divinorum!<BR>
      <FONT COLOR="#FF0000"><BIG>WARNING...EXTREMELY POTENT!</BIG></FONT></B>
      <P>
      <P>
   

In [123]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")         #Printing first 1000 characters of a spam email after converting to plain text

<
GET HIGH...LEGALLY!
IT REALLY WORKS!
PASSES ALL DRUG TESTS!
EXTREMELY POTENT!
CLICK HERE for more
      info on Salvia Divinorum
CLICK HERE for SALVIA
      5X EXTRACT!
WARNING...VERY POTENT!
CLICK HERE for SALVIA
      13X
. The most POTENT, LEGAL, SMOKABLE herb on the planet! 13 times the
      power of Salvia Divinorum!
WARNING...EXTREMELY POTENT!
Removal Information:
We are strongly against sending unsolicited emails to those who do not wish
      to receive our special mailings. You have opted in to one or more of our
      affiliate sites requesting to be notified of any special offers we may run
      from time to time. This is NOT unsolicited email. If you do not wish to receive
      further mailings, please
click here to be removed
      from the list
. Please accept our apologies if you have been sent this
      email in error. We honor all removal requests within 24 hours. ...


In [124]:
#Writing a function which takes email as input and outputs its content as plain text - Not all email is having html tags so converting only htmls to plain text here
def email_to_text(email):
    html=None
#The walk() method is an all-purpose generator which can be used to iterate over all the parts and subparts of a message object tree, in depth-first traversal order
    for part in email.walk():
        ctype=part.get_content_type()
        if not ctype in ('text/plain','text/html'):
            continue
        try:
            content=part.get_content()
        except:
            content=str(part.get_payload())
        if ctype=='text/plain':
            return content
        else:
            html=content
    if html:
        return html_to_plain_text(html)

In [125]:
print(email_to_text(sample_html_spam)[:100], "...")              #The first 100character of the output of one last cell should be same as this

<
GET HIGH...LEGALLY!
IT REALLY WORKS!
PASSES ALL DRUG TESTS!
EXTREMELY POTENT!
CLICK HERE for more
 ...


In [126]:
#Introducting Stemming to make the word counting efficient, so that it counts the variations of the same word under one bracket
import nltk

stemmer=nltk.PorterStemmer()
for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, "=>", stemmer.stem(word))

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


In [127]:
pip install nltk

  Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached regex-2021.10.21-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dc-workspace 1.0.0 requires jupytext, which is not installed.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [128]:
#Removing Stopwords -
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [129]:
#stop_words

In [130]:
pip install urlextract

  Using cached urlextract-1.4.0-py3-none-any.whl (20 kB)
  Using cached uritools-3.0.2-py3-none-any.whl (12 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached filelock-3.3.1-py3-none-any.whl (9.7 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dc-workspace 1.0.0 requires jupytext, which is not installed.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [131]:
#Used for extracting all the urls of the email, so that we can remove them when building our word count matrix
import urlextract

url_extractor=urlextract.URLExtract()

In [132]:
#Now that we have extracted all the data from email - lets break down the sentences into word and check the frequency of each word
import re
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransform(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True, replace_urls=True, replace_numbers=True, stemming=True, remove_stopwords=True):
        self.strip_headers=strip_headers
        self.lower_case=lower_case
        self.remove_punctuation=remove_punctuation
        self.replace_urls=replace_urls
        self.replace_numbers=replace_numbers
        self.stemming=stemming
        self.remove_stopwords=remove_stopwords
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed=[]
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text=text.lower()
            if self.replace_urls and url_extractor is not None:
                urls=list(set(url_extractor.find_urls(text)))           #Set function returns an empty set if no element is passed. Non-repeating element iterable modified as passed as argument.
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text=text.replace(url, ' URL ')
            if self.replace_numbers:
                text=re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?','NUMBER', text)
            if self.remove_punctuation:
                text=re.sub(r'\W+',' ',text,flags=re.M)
            if self.remove_stopwords:
                words=text.split()
                words_without_sw=[word for word in words if not word in stop_words]
            word_counts=Counter(words_without_sw)
            if self.stemming and stemmer is not None:
                stemmed_word_counts=Counter()
                for word,count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)    
        return np.array(X_transformed)

In [133]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransform().fit_transform(X_few)
X_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'christian': 3, 'jefferson': 2, 'superstit': 2, 'one': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'find': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'effect': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'becom': 1, 'pervert': 1, 'system': 1, 'ever': 1, 'shone': 1, 'man': 1, 'absurd': 1, 'untruth': 1, 'perpetr': 1, 'upon': 1, 'larg': 1, 'band': 1, 'dupe': 1, 'import': 1, 'led': 1, 'paul': 1, 'first': 1, 'great': 1, 'corrupt': 1}),
       

In [134]:
#Creating vocabulary as a feature matrix
from scipy.sparse import csr_matrix

class WordCounterToVectorTransform(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size=vocabulary_size
    def fit(self,X,y=None):
        total_count=Counter()                                         
        for word_count in X:                              
            for word, count in word_count.items():
                total_count[word] += min(count, 10)                     #Finding out common words which has occurances less than equal to 10
        most_common=total_count.most_common()[:self.vocabulary_size]    #Finding out the most common words to build the vocabulary array
        self.most_common_=most_common                                   #Assigning to an instance of the class
        self.vocabulary_={word: index + 1 for index, (word, count) in enumerate(most_common)}      #Setting up the vocab - it will have a combination of word & index
        print(self.vocabulary_)                                         #Assigning to another instance of the class
        return self
    def transform(self,X,y=None):
        rows=[]
        cols=[]
        data=[]
        for row, word_count in enumerate(X):                          
            for word, count in word_count.items():                   #Checking the number of words present in the each email
                rows.append(row)                                     #Building the rows of the matrix
                cols.append(self.vocabulary_.get(word, 0))           #Storing the index of the word present in the vocabulary to be used to build csr matrix
                data.append(count)
            #print(rows[7:])
            #print(cols[7:])
            #print(data)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [135]:
vocab_transformer = WordCounterToVectorTransform(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

{'url': 1, 'christian': 2, 'group': 3, 'wrote': 4, 'jefferson': 5, 'superstit': 6, 'one': 7, 'half': 8, 'rogueri': 9, 'teach': 10}


<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [136]:
X_few_vectors.toarray()

array([[ 5,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0],
       [64,  1,  3,  0,  0,  2,  2,  2,  2,  2,  2],
       [40,  4,  0,  3,  1,  0,  0,  0,  0,  0,  0]])

In [137]:
vocab_transformer.vocabulary_

{'url': 1,
 'christian': 2,
 'group': 3,
 'wrote': 4,
 'jefferson': 5,
 'superstit': 6,
 'one': 7,
 'half': 8,
 'rogueri': 9,
 'teach': 10}

In [138]:
#Now applying the 2 functions written above to transform all the email data
from sklearn.pipeline import Pipeline

preprocess_pipeline=Pipeline([
    ('email_to_word_count',EmailToWordCounterTransform()),
    ('wordcount_to_vector',WordCounterToVectorTransform()),
])

X_train_transformed=preprocess_pipeline.fit_transform(X_train)

{'number': 1, 'url': 2, 'list': 3, 'use': 4, 'get': 5, 'mail': 6, 'one': 7, 'time': 8, 'like': 9, 'messag': 10, 'com': 11, 'email': 12, 'would': 13, 'work': 14, 'make': 15, 'new': 16, 'peopl': 17, 'free': 18, 'user': 19, 'net': 20, 'date': 21, 'wrote': 22, 'rpm': 23, 'want': 24, 'linux': 25, 'also': 26, 'spamassassin': 27, 'file': 28, 'go': 29, 'need': 30, 'year': 31, 'way': 32, 'think': 33, 'group': 34, 'tri': 35, 'look': 36, 'receiv': 37, 'know': 38, 'us': 39, 'e': 40, '_______________________________________________': 41, 'run': 42, 'inform': 43, 'even': 44, 'spam': 45, 'said': 46, 'could': 47, 'first': 48, 'say': 49, 'see': 50, 'chang': 51, 'right': 52, 'problem': 53, 'exmh': 54, 'pleas': 55, 'world': 56, 'line': 57, 'day': 58, 'thing': 59, 'take': 60, 'may': 61, 'send': 62, 'money': 63, 'system': 64, 'remov': 65, 'find': 66, 'good': 67, 'much': 68, 'well': 69, 'name': 70, 'compani': 71, 'call': 72, 'mani': 73, 'help': 74, 'talk': 75, 'servic': 76, 'subject': 77, 'packag': 78, 'add

In [146]:
#Model the preprocessed data using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

spam_clf=LogisticRegression(solver='liblinear',random_state=42)
score=cross_val_score(spam_clf,X_train_transformed,y_train,cv=3,verbose=3)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ................................ score: (test=0.882) total time=   0.2s
[CV] END ................................ score: (test=0.882) total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] END ................................ score: (test=0.897) total time=   0.1s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s finished


In [112]:
score.mean()

0.8874999999999998

In [147]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf = LogisticRegression(solver="liblinear", random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))

Precision: 89.92%
Recall: 96.27%


In [96]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[463,  19],
       [ 51,  67]])

In [139]:
#Model the preprocessed data using SVM
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc_clf=SVC()
score=cross_val_score(svc_clf,X_train_transformed,y_train,cv=3,verbose=3)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ................................ score: (test=0.841) total time=   2.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


[CV] END ................................ score: (test=0.841) total time=   3.7s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.4s remaining:    0.0s


[CV] END ................................ score: (test=0.840) total time=   3.8s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.2s finished


In [140]:
score.mean()

0.8408333333333333

In [141]:
from sklearn.metrics import precision_score, recall_score

svm_clf = SVC()
svm_clf.fit(X_train_transformed, y_train)

y_pred_svm = svm_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred_svm)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred_svm)))

Precision: 80.33%
Recall: 100.00%


In [142]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_svm)

array([[  0, 118],
       [  0, 482]])

In [143]:
#print(y_test)

[1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1
 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1
 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0
 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1
 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1
 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1
 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 0 1 1 1 1 

In [1]:
#print(y_pred)